# Markov Chains (or stochastic recurrence relations)

Some fixed order dynamic systems are naturally *stochastic* (or - in other words - involve a bit of randomness)

- but one which is naturally *stochastic*.  In this Subsection we introduce the basic version of such a model via its most popular application - as a model of written text.  This kind of dynamic system is often referred to as a *Markov Chain*.

In [2]:
## This code cell will not be shown in the HTML version of this notebook
# imports from custom library for animations
import sys
sys.path.append('../../')
from demo_libraries import markov_chains_library as marklib

# import standard libs
import numpy as np
import pandas as pd

# path to data
datapath = '../../datasets/markov_chains/'

# This is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

For example, take the simple sentence shown in the animation below "my dog runs".  Each *word* in this sentence does - intuitively - seem to follow its predecessor in an orderly, functional, and predictable way.  In the animation below we walk through the sentence word-by-word, highlighting the fact that each word follows its immediate predecessor just like a $D=1$ dynamic system.  

<figure>
<p>
<img src= 'images/dog1.gif' width="50%" height="50%" alt=""/>
</p>
<figcaption> <strong>Figure 2:</strong> <em> Each word in a simple English sentence follows its predecessor in an orderly, functional, and predictive fashion - much like an order $D=1$ dynamic system.
</em>
</figcaption>
</figure>

This kind of intuitive leap makes sense more generally as well.  We can also think of each word in a sentence following logically based not just on its immediate predecessor, but several words just like an order $D$ dynamic system.  Text certainly is generally structured like this - with preceding words in sentence determining those that follow. 

However while text certainly seems to have the structure of a dynamic system like the ones we have seen thus far, it does have one attribute that we have not seen thus far: choice. A given word does not always completely determine the word that follows it.   For example, in the sentence above we could imagine a range of words following the word "dog" instead of "runs", like e.g., "eats".  This would give us the sentence "my dog eats" instead of "my dog runs", which is a perfectly valid and meaningful English sentence.  Of course there are other words that could follow "dog" as well, some of which like e.g., the word "sighs" that while valid would be less common than "eats".  However some words, like e.g., "dracula", making the sentence "my dog dracula" do not make any sense at all.

<figure>
<p>
<img src= 'images/dog2.gif' width="50%" height="50%" alt=""/>
</p>
<figcaption> <strong>Figure 3:</strong> <em> Many words could potentially follow the word "dog" in this sentence.  However some words - like "runs" or "eats" - are far more likely than others like "sighs".  Further, some words like e.g., "dracula" do not make any sense.
</em>
</figcaption>
</figure>

So in summary, while a valid sentence of English words clearly has *ordered* structure like a dynamic system, with each word following its predecessor, there are often many *choices* for subsequent words (unlike the dynamic systems we have seen previously).  Moreover of the choices following a particular word we can reasonably say that some are more likely to occur than others.  In other words, these choices are *stochastic* in nature (*stochastic* means *choices*) - with each having a different probability of occurring.  For example, below we show the four choices of words following "dog" in the sentence above assigning (using our intuition) a probability of occurring to each.  Here we suppose the word "runs" is the most probable, "eats" the next, and so on.  The probabilities shown do not add up to $1$ because there could be other reasonable words that could follow "dog" other than those shown here.  In other words, with this order one dynamic system model of text we have is a *histogram* (a discrete probability distribution) over all the possible words that follow 'dog' (as well as any other word).

<figure>
<p>
<img src= 'images/dog3.png' width="60%" height="60%" alt=""/>
</p>
<figcaption> <strong>Figure 4:</strong> <em> Here we show the selection of words from Figure 3 that could possibly follow the word "dog", along with a probability $p$ of the word occurring (which here we set ourselves via intuition).  In this case the word "eats" has the highest probability of occurring after "dog", then "runs", and so on.  The probabilities do not add up to $1$ because there are other words that could reasonably follow "dog" that we do not list here.
</em>
</figcaption>
</figure>

Note how we could just as well model text as a general order $D$ model in terms of its words, in the same manner described above for the case of $D = 1$.  For example below we illustrate an order $D = 2$ word model of the same sentence above.  

<figure>
<p>
<img src= 'images/dog_with_hist.png' width="80%" height="60%" alt=""/>
</p>
<figcaption> <strong>Figure 5:</strong> <em> (left panel) Here we show the selection of words from Figure 3 that could possibly follow the word "dog", along with a probability $p$ of the word occurring (which here we set ourselves via intuition).  In this case the word "eats" has the highest probability of occurring after "dog", then "runs", and so on.  The probabilities do not add up to $1$ because there are other words that could reasonably follow "dog" that we do not list here. (right panel) The set of next possible words and their discrete probabilities viewed as a *histogram* or discrete probability distribution.
</em>
</figcaption>
</figure>

To more accurately estimate these probabilities - and to identify all of the other possible words that follow "dog" - it is common to take a large body of text (like e.g., a long book), scan it for all occurrences of the word "dog" and make a listing of all the words following it along with their frequency.  Then in order to create a sample probability for each - so that the sum of the probabilities adds up to $1$ - we simply divide off the total number of times "dog" appeared in the text (often referred to as a *corpus*).

Irregardless of how we compute the probabilities, mathematically speaking we codify the list possible outputs and their probability of occurrence - like the list of possible words shown above following "dog" - as a *histogram* (which is often also called a *probability mass function*).  This is just a mathematical function with many possible outputs to a single input, each weighted with its respective probability $\mathscr{p}$.  For example, we can formally write the histogram of possible outputs of words following "dog" as

\begin{equation}
\text{histogram}\left(\text{"dog"}\right) = 
\begin{cases}
\text{"runs"} \,\,\,\,\,\,\,\,\,\,\,\, \text{with probability} \,\,\,\mathscr{p} = 0.4 \\
\text{"eats"} \,\,\,\,\,\,\,\,\,\,\,\,\, \text{with probability} \,\,\,\mathscr{p} = 0.3 \\
\text{"sighs"} \,\,\,\,\,\,\,\,\,\, \text{with probability} \,\,\,\mathscr{p} = 0.05 \\
\text{"dracula"} \,\,\,\,\, \text{with probability} \,\,\,\mathscr{p}= 0 \\
\vdots
\end{cases}
\end{equation}

Here in each case '$p$' stands for the *probability* of the corresponding word occurring.  Our prediction for the next word in the sentence is then simply the one that is most valuable, or the one with the largest probability.  Here suppose that word is "runs".  Translating this statement into math, we predict the word following "dog" by taking the $\text{argmax}$ over all of the choices above as

\begin{equation}
\text{(word we predict to follow "dog")} \,\,\,\,\, \text{"runs"} = \underset{\mathscr{p}}{\text{argmax}}\,\,\,\, \text{historgram}\left(\text{"dog"}\right).
\end{equation}

More generally, if we denote by $x_{t}$ the $t^{th}$ word in a sentence, and $u$ any particular word with the probability $\mathscr{p}$ after of $x_{t}$ (often called the *transition probability*) than the histogram can be written more generally as

\begin{equation}
\text{histogram}\left(x_{t}\right) = 
u \,\,\,\,\text{with probability} \,\,\,\mathscr{p}.
\end{equation}

Then the choice of the next word $x_{t+1}$ can likewise be written in general as

\begin{equation}
x_{t+1} = \underset{\mathscr{p}}{\text{argmax}}\,\, \text{histogram}\left(x_{t}\right).
\end{equation}

We can perhaps now see how this model of written text is indeed a dynamic system.  If we in general denote 

\begin{equation}
\underset{\mathscr{p}}{\text{argmax}}\,\, \text{histogram}\left(x_{t}\right) = f\left(x_{t}\right) 
\end{equation}

and the first word in a sentence as $x_1 = \gamma$.  Then the order $D = 1$ model of text we have detailed above does indeed fit the pattern of a *recurrence relation* as

\begin{array}
\
x_1 = \gamma \\
x_{t+1} = f\left(x_{t}\right).
\end{array}

We could then likewise define general order $D$ model of text as well.  This would work precisely as we saw above but - as illustrated below for an example where $D = 2$ - instead of one word from the past we use two, making a prediction of the word that follows this pair precisely as we did above (i.e., by forming a histogram / discrete probability distribution of all words that can follow such a pair in a large corpus, and choosing the word with the highest probability / likeliness of occuring).

<figure>
<p>
<img src= 'images/order_2.png' width="80%" height="80%" alt=""/>
</p>
<figcaption> <strong>Figure 6:</strong> <em> An illustration of an order $D = 2$ Markov chain for modeling text.  Here we use a pair of words and make a prediction about the word following them precisely as we did in the order $D = 1$ case - by forming a historgram / discrete probability distribution over all the words following this pair seen in a large corpus.  Our prediction is then once again the word with the largest value in this histogram / highest discrete probability of occuring.  In this toy example that word is "short".
</em>
</figcaption>
</figure>

Formally we can express a general order $D$ dynamic system of this form precisely as we did above for the order $D = 1$ case, via the general update step

\begin{equation}
x_{t+1} = f\left(x_{t},...,x_{t-\mathcal{O}+1}\right).
\end{equation}

Here the only difference is that now the function $f$ involves a histogram over the prior words $x_{t},...,x_{t-\mathcal{O}+1}$, i.e.,

\begin{equation}
\underset{\mathscr{p}}{\text{argmax}}\,\, \text{histogram}\left(x_{t},...,x_{t-\mathcal{O}+1}\right) = f\left(x_{t},...,x_{t-\mathcal{O}+1}\right) 
\end{equation}

#### <span style="color:#a50e3e;">Example 6. </span>  Generating text word-by-word via a Markov chain

In this example we generate a Markov chain model of text using the classic novel *War of the Worlds* by H.G. Wells to define our transition probabilities.  Below we print out the first $500$ characters of the novel.  Note some pre-processing has been done here - in particular we removed any strange characters introduced when converting this book to its e-version, and lower-case-afied all alphabetical characters.

In [28]:
## This code cell will not be shown in the HTML version of this notebook
csvname = datapath + "war_of_the_worlds.txt"
model = recurlib.word_level_markov_model.Markov(csvname)
model.text[:500]

"the war of the worlds  by h. g. wells   but who shall dwell in these worlds if they be  inhabited? are we or they lords of the  world? and how are all things made for man?    kepler (quoted in the anatomy of melancholy)  book one  the coming of the martians  chapter one  the eve of the war no one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than man's and yet as mortal as his own; that as men busied "

Using an order $D = 1$ model we then pick a word randomly from the text, and start generating text.  Below we compare a chunk of $30$ words from the text following this initial input, and below it we show the result of the Markov model.  Here the input word $x_1 = \gamma$ is colored red, and the $30$ words generated using it are colored blue.  Note this means that we first plug in the word "had" into our model, which returns the word "been".  Then we return "been" and it returns "the", etc.,

In [10]:
## This code cell will not be shown in the HTML version of this notebook
order = 1; num_words = 30;
demo = recurlib.markov_words_demo.show_order(csvname,order,num_words)

-------- TRUE TEXT -------
had done for countless years as though no planet mars existed in the sky even at woking station and horsell and chobham that was the case in woking junction until late


-------- ORDER = 1 MODEL TEXT -------
had been the martians were the martians were the martians were the martians were the martians were the martians were the martians were the martians were the martians were the martians


Clearly we can see that the Markov model, having only a single word in the past to base the next word on, does not generate anything meaningful.  However as we increase the order to e.g., $D = 2$ we can see that the generated sentence starts to make more sense, matching its original as shown below.  Here the two initial words are colored red, with the remaining generated words colored blue.  Notice this means that we first plug in the first two words (here the phrase "trucks bearing") and it returns "huge", then we plug in the next two words (here "bearing huge") and it returns "guns", etc.,

In [12]:
## This code cell will not be shown in the HTML version of this notebook
order = 2; num_words = 30;
demo = recurlib.markov_words_demo.show_order(csvname,order,num_words)

-------- TRUE TEXT -------
trucks bearing huge guns and carriages crammed with soldiers these were the guns that were brought up from woolwich and chatham to cover kingston there was an exchange of pleasantries you ll


-------- ORDER = 2 MODEL TEXT -------
trucks bearing huge guns and such morbidities never enter the scheme of their houses astonished how are all things made for the most part the daily telegraph for instance in the darkness


As we increase the order $D$ of the model the generated text will begin to match the original more and more.  For example, by the time we crank up the order to $D = 10$ the text generated by the model is identical to the original.

In [13]:
## This code cell will not be shown in the HTML version of this notebook
order = 10; num_words = 30;
demo = recurlib.markov_words_demo.show_order(csvname,order,num_words)

-------- TRUE TEXT -------
we hurried along and the artilleryman talked in whispers and looked now and again over our shoulders once or twice we stopped to listen after time we drew near the road and as we did so we heard the clatter


-------- ORDER = 10 MODEL TEXT -------
we hurried along and the artilleryman talked in whispers and looked now and again over our shoulders once or twice we stopped to listen after time we drew near the road and as we did so we heard the clatter


Why does this happen?  Notice that when we increase the order of the model the number of unique input sequences proliferates rapidly.  Eventually, when we increase the order enough, there remains only a single exemplar (input/output pair) in the text to construct each associated histogram (i.e., every input sequence used to determine the transition probabilities is *unique*).  Past this point we only have a single example of each input, thus only one choice for its associated output: whatever follows it in the text (with probability $\mathscr{p} = 1$).

---

The same thought process leading to the word-by-word model of text detailed above also leads to a similar idea: modeling text *character-by-character*.  All of the same logic developed for the word-wise model applies here as well.  If we examine a string of English text - on the level of characters this time - once again upon reflection we see a natural ordering to the characters (as in the example shown in the Figure below).  Some characters - as with words - seem to naturally follow others.  Of course just as with words, the issue here is once again at each step we have *stochasticity* or multiple choices for what the next character should be.  In other words, we have a histogram / discrete probability distribution over the full set of possible English characters that reflects the likelihood of each character occurring (with respect to a large corpus on which we build such histograms).  As we increase the order of our model - just as in the word-wise case - our generated text looks more and more like the corpus on which we compute our histograms.

<figure>
<p>
<img src= 'images/chars.png' width="70%" height="50%" alt=""/>
</p>
<figcaption> <strong>Figure 7:</strong> <em> An illustration of an order $D = 1$ (top), $D = 2$ (middle), and $D = 3$ order Markov chain for modeling text *character-wise*.  In each case to make a reasonable prediction about the next character we form a histogram / discrete probability distribution over all the characters that follow each input character(s) as seen in a large corpus.  Our prediction is then once again the character with the largest value in this histogram / highest discrete probability (note in the order $D = 2$ case shown here the empty circle contains the 'space' character).  As we increase the order here - just as with the word-wise model detailed above - our generated sequence will start to look more and more like the text on which we computed our histograms.
</em>
</figcaption>
</figure>

#### <span style="color:#a50e3e;">Example 7. </span>  Generating text character-by-character using a Markov chain

Just modeled text by *words* above using a Markov chain, we can likewise model it via *characters*.  For the same intuitive reasons as discussed in the context of the text-wise modeling scheme - characters often logically follow one another in succession - we can model text as a stochastic dynamic system (a Markov chain) over characters as well.  

Below we show the result of an order $D = 1$ Markov chain model using the characters instead of words, and the same text (H.G. Well's classic *War of the Worlds*) to calculate our transition probabilities.  Of course using only a single character as precedent we cannot capture much about the text, as reflected in the generated text below.  Here the single character used as input is colored red, with $300$ generated characters using the order $1$ model colored blue.  Note that this means that the first character "n" is plugged into the model and returns the second, here the space character.  This is then plugged in to generate "t", etc.,

In [25]:
## This code cell will not be shown in the HTML version of this notebook
order = 1; num_chars = 300;
demo = recurlib.markov_chars_demo.show_order(csvname,order,num_chars)

-------- TRUE TEXT -------
now at a pace of many miles a second through the empty gulf of space, hour by hour and day by day, nearer and nearer. it seems to me now almost incredibly wonderful that, with that swift fate hanging over us, men could go about their petty concerns as they did.  i remember how jubilant markham was at


-------- ORDER = 1 MODEL TEXT -------
n the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the


As we saw in the previous example, as we increase the order $D$ of the model we capture more and more about the text, and can therefore generate more and more meaningful sentences.  For example, below we show the result of an order $D = 5$ model, comparing to the similar chunk of the true text.  With this many characters we actually start to generate a number of real words.

In [26]:
## This code cell will not be shown in the HTML version of this notebook
order = 5; num_chars = 300;
demo = recurlib.markov_chars_demo.show_order(csvname,order,num_chars)

-------- TRUE TEXT -------
dow from which we had watched the martians, and went very quietly downstairs.  the artilleryman agreed with me that the house was no place to stay in.  he proposed, he said, to make his way londonward, and thence rejoin his battery no. , of the horse artillery.  my plan was to return at once to leatherhe


-------- ORDER = 5 MODEL TEXT -------
dow from the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham before the street cobham bef


Increasing the order $D = 10$ we can further observe this trend.

In [27]:
## This code cell will not be shown in the HTML version of this notebook
order = 10; num_chars = 300;
demo = recurlib.markov_chars_demo.show_order(csvname,order,num_chars)

-------- TRUE TEXT -------
from which it had been strained. the vapour did not diffuse as a true gas would do.  it hung together in banks, flowing sluggishly down the slope of the land and driving reluctantly before the wind, and very slowly it combined with the mist and moisture of the air, and sank to the earth in the form of dust. s


-------- ORDER = 10 MODEL TEXT -------
from which the martians were setting fire to everything was to be done.  we're under! we're beat."  "but if that is so, what is the essence of the pit.  i saw an abandoned arguments.  it was a lieutenant.  "what confounded nonsense!"  "you'll hear more of elevation of the great majority of people were haggard


Finally, just as in the word generating case, if we increase the order $D$ past a certain point our model will generate the text exactly.  Below we show the result of an order $D = 50$ model, which generates precisely the true text shown above it.  This happens for exactly the same reasoning given previously in the context of the word based model: as we increase the order of the model the number of unique input sequences balloons rapidly, until each input sequence of the text is *unique*.  This means that there is only one example of each used to determine the transition probabilities, i.e., precisely the one present in the text.

In [24]:
## This code cell will not be shown in the HTML version of this notebook
order = 50; num_chars = 300;
demo = recurlib.markov_chars_demo.show_order(csvname,order,num_chars)

-------- TRUE TEXT -------
tte of maybury hill, with its tree-tops and roofs black and sharp against the red.  even as i beheld this a lurid green glare lit the road about me and showed the distant woods towards addlestone.  i felt a tug at the reins.  i saw that the driving clouds had been pierced as it were by a thread of green fire, suddenly lighting their confusion and f


-------- ORDER = 50 MODEL TEXT -------
tte of maybury hill, with its tree-tops and roofs black and sharp against the red.  even as i beheld this a lurid green glare lit the road about me and showed the distant woods towards addlestone.  i felt a tug at the reins.  i saw that the driving clouds had been pierced as it were by a thread of green fire, suddenly lighting their confusion and f


## 15.3.3  Fixed order and limited memory

The consequence of such systems being limited by their order is perhaps most clearly seen by a simple example of a Markov chain model of text.  For example, suppose we have constructed a word-based Markov model of order $D = 1$, whose transition probabilities have been determined using a large text corpus.  We the apply our model to both of the sentences shown below, to predict the word following "is".

<figure>
<p>
<img src= 'images/dog4.png' width="60%" height="60%" alt=""/>
</p>
<figcaption> <strong>Figure 5:</strong> <em>  The main shortcoming of fixed order systems is exemplified in this toy example.  Here we suppose we have an order $D = 1$ model whose transition probabilities have been determined on a large training corpus.  Here we use our order $D = 1$ to predict the next word of each setnence, following the word "is".  However since the model is order $D = 1$ the *same* word will be predicted for each sentence.  Given the different subject / context of each, this will likely mean that at least one of the sentences will not make sense.
</em>
</figcaption>
</figure>

The problem here is that - because we have used an order $D = 1$ model - the *same* word will be predicted to follow the word "is" in both sentences.  This will likely mean that at least one (if not both) of the completed sentences will not make sense, since they have completely different subjects.  Because a fixed order dynamic system is limited by its order, and cannot use any information from earlier in a sequence, this problem can arise regardless of the order $D$ we choose.